## Segmenting and Clustering Neighborhoods in Toronto

<div class="alert alert-block alert-info" style="background-color: #E0F0FE;margin-top: 20px">

<font size = 3>

## Table of contents
* [1. Create a Notebook and Import Libraries](#create)
* [2. Scrape the Wikipedia Page](#scrap)
* [3. Transform the Data](#transform)
* [4. Add Coordinates](#add)
* [5. Explore and Cluster the Neighborhoods in Toronto](#explore)


### 1. Create a Notebook and Import Libraries <a name="create"></a>

In [1]:
# importing libraries
import pandas as pd
import numpy as np
import requests
from bs4 import BeautifulSoup

### 2. Scrape the Wikipedia Page  <a name="scrap"></a>

In [2]:
# getting data
data= requests.get('https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M').text

# using beautiful soup to parse data.
soup = BeautifulSoup(data,'lxml')

#print(soup.prettify())

### 3. Transform the Data  <a name="transform"></a>

In [3]:
# getting table data

My_table = soup.find('table',{'class':'wikitable sortable'})
links = My_table.findAll('tr')
#links

In [4]:
#Creating dataframe

data = []

for row in links:
    data.append([t.text.strip() for t in row.find_all('td')])

df = pd.DataFrame(data, columns=['PostalCode', 'Borough', 'Neighbourhood'])

#Remove null postal codes if any

df = df[~df['PostalCode'].isnull()]

print(df.head(5))

  PostalCode           Borough               Neighbourhood
1        M1A      Not assigned                            
2        M2A      Not assigned                            
3        M3A        North York                   Parkwoods
4        M4A        North York            Victoria Village
5        M5A  Downtown Toronto  Regent Park / Harbourfront


In [5]:
# Check dataframe
df.info()
df.shape

<class 'pandas.core.frame.DataFrame'>
Int64Index: 180 entries, 1 to 180
Data columns (total 3 columns):
PostalCode       180 non-null object
Borough          180 non-null object
Neighbourhood    180 non-null object
dtypes: object(3)
memory usage: 5.6+ KB


(180, 3)

#### Ignore cells with a non assigned borough.

In [6]:
# Ignore cells with a non assigned borough
df.drop(df[df['Borough']=="Not assigned"].index,axis=0, inplace=True)
df.head()

,PostalCode,Borough,Neighbourhood
3,M3A,North York,Parkwoods
4,M4A,North York,Victoria Village
5,M5A,Downtown Toronto,Regent Park / Harbourfront
6,M6A,North York,Lawrence Manor / Lawrence Heights
7,M7A,Downtown Toronto,Queen's Park / Ontario Provincial Government


In [7]:
#reindex and check dataframe
df1 = df.reset_index()
df1.info()
df1.shape

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 103 entries, 0 to 102
Data columns (total 4 columns):
index            103 non-null int64
PostalCode       103 non-null object
Borough          103 non-null object
Neighbourhood    103 non-null object
dtypes: int64(1), object(3)
memory usage: 3.3+ KB


(103, 4)

#### Combine neighborhoods with same postal code and separated with a comma.

In [8]:
# combine rows with same postal code as needed
df2= df1.groupby('PostalCode').agg(lambda x: ','.join(x))

# Separate neighborhoods in same postal code with ',' instead of '/'
df3 = df2.apply(lambda x: x.str.replace(" /",","))

df3.head()

,Borough,Neighbourhood
PostalCode,,
M1B,Scarborough,"Malvern, Rouge"
M1C,Scarborough,"Rouge Hill, Port Union, Highland Creek"
M1E,Scarborough,"Guildwood, Morningside, West Hill"
M1G,Scarborough,Woburn
M1H,Scarborough,Cedarbrae


#### If a cell has a borough but not assigned a neighborhood, then the neighborhood will be the same as the borough.

In [9]:
df3.loc[df3['Neighbourhood']=="Not assigned",'Neighbourhood']=df3.loc[df3['Neighbourhood']=="Not assigned",'Borough']

#Sorting Alphabetically by Postal Code

df3.sort_values(by=['PostalCode'], ascending = True)

df3.head()

,Borough,Neighbourhood
PostalCode,,
M1B,Scarborough,"Malvern, Rouge"
M1C,Scarborough,"Rouge Hill, Port Union, Highland Creek"
M1E,Scarborough,"Guildwood, Morningside, West Hill"
M1G,Scarborough,Woburn
M1H,Scarborough,Cedarbrae


In [10]:
#reindex and check dataframe
df4 = df3.reset_index()
df4.info()
df4.shape

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 103 entries, 0 to 102
Data columns (total 3 columns):
PostalCode       103 non-null object
Borough          103 non-null object
Neighbourhood    103 non-null object
dtypes: object(3)
memory usage: 2.5+ KB


(103, 3)

### 4. Add Coordinates  <a name="add"></a>

#### Get Coordinates data from the csv file.

In [11]:
Geo = pd.read_csv("https://cocl.us/Geospatial_data/geopy.csv")
Geo.head()

,Postal Code,Latitude,Longitude
0,M1B,43.806686,-79.194353
1,M1C,43.784535,-79.160497
2,M1E,43.763573,-79.188711
3,M1G,43.770992,-79.216917
4,M1H,43.773136,-79.239476


#### Add Coordinates to the dataframe.

In [12]:
Geo.rename(columns={'Postal Code': 'PostalCode'}, inplace=True)
df5 = pd.merge(df4, Geo, how='right', left_on='PostalCode', right_on='PostalCode')
df5.head()

,PostalCode,Borough,Neighbourhood,Latitude,Longitude
0,M1B,Scarborough,"Malvern, Rouge",43.806686,-79.194353
1,M1C,Scarborough,"Rouge Hill, Port Union, Highland Creek",43.784535,-79.160497
2,M1E,Scarborough,"Guildwood, Morningside, West Hill",43.763573,-79.188711
3,M1G,Scarborough,Woburn,43.770992,-79.216917
4,M1H,Scarborough,Cedarbrae,43.773136,-79.239476


In [13]:
#Checking dataframe
df5.info()
df5.shape

<class 'pandas.core.frame.DataFrame'>
Int64Index: 103 entries, 0 to 102
Data columns (total 5 columns):
PostalCode       103 non-null object
Borough          103 non-null object
Neighbourhood    103 non-null object
Latitude         103 non-null float64
Longitude        103 non-null float64
dtypes: float64(2), object(3)
memory usage: 4.8+ KB


(103, 5)

### 5. Explore and Cluster the Neighborhoods in Toronto  <a name="explore"></a>

#### Download all needed dependencies.

In [3]:
#!conda install -c conda-forge geopy --yes
#!conda install -c conda-forge folium=0.5.0 --yes

In [14]:
import numpy as np # library to handle data in a vectorized manner

import pandas as pd # library for data analsysis
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

import json # library to handle JSON files

from geopy.geocoders import Nominatim # convert an address into latitude and longitude values

import requests # library to handle requests
from pandas.io.json import json_normalize # tranform JSON file into a pandas dataframe

# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors

# import k-means from clustering stage
from sklearn.cluster import KMeans

import folium # map rendering library

print('Libraries imported.')

Libraries imported.


In [15]:
print('The dataframe has {} boroughs and {} neighborhoods.'.format(len(df5['Borough'].unique()),df5.shape[0]))

The dataframe has 10 boroughs and 103 neighborhoods.


In [16]:
address = 'Toronto, ON'

geolocator = Nominatim(user_agent="ny_explorer")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinate of Toronto are {}, {}.'.format(latitude, longitude))

The geograpical coordinate of Toronto are 43.6534817, -79.3839347.


#### Create map of Toronto and its neighborhoods.

In [17]:
# create map of Toronto using latitude and longitude values
map_Toronto = folium.Map(location=[latitude, longitude], zoom_start=10)

# add markers to map
for lat, lng, borough, neighborhood in zip(df5['Latitude'], df5['Longitude'], df5['Borough'], df5['Neighbourhood']):
    label = '{} - {}'.format(neighborhood, borough)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_Toronto)  
    
map_Toronto

#### Define Foursquare Credentials and Version.

Credentials <br>
CLIENT_ID = 'xyz' # your Foursquare ID <br>
CLIENT_SECRET = 'xyz' # your Foursquare Secret <br>
VERSION = '20180605' # Foursquare API version <br>

#### Explore the first neighborhood in our dataframe.

In [19]:
#get the neighbourhood name
df5.loc[0, 'Neighbourhood']

'Malvern, Rouge'

In [20]:
#get the neighborhood coordinates
neighborhood_latitude = df5.loc[0, 'Latitude'] # neighborhood latitude value
neighborhood_longitude = df5.loc[0, 'Longitude'] # neighborhood longitude value

neighborhood_name = df5.loc[0, 'Neighbourhood'] # neighborhood name

print('Latitude and longitude values of {} are {}, {}.'.format(neighborhood_name, 
                                                               neighborhood_latitude, 
                                                               neighborhood_longitude))

Latitude and longitude values of Malvern, Rouge are 43.806686299999996, -79.19435340000001.


#### Get the top 100 venues that are in Marble Hill within a radius of 500 meters.

In [21]:
LIMIT = 100 # limit of number of venues returned by Foursquare API
radius = 500 # define radius
url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
    CLIENT_ID, 
    CLIENT_SECRET, 
    VERSION, 
    neighborhood_latitude, 
    neighborhood_longitude, 
    radius, 
    LIMIT)

In [22]:
results = requests.get(url).json()
#results

In [23]:
# function that extracts the category of the venue
def get_category_type(row):
    try:
        categories_list = row['categories']
    except:
        categories_list = row['venue.categories']
        
    if len(categories_list) == 0:
        return None
    else:
        return categories_list[0]['name']

#### Clean the json and structure it into a *pandas* dataframe.

In [24]:
venues = results['response']['groups'][0]['items']
    
nearby_venues = json_normalize(venues) # flatten JSON

# filter columns
filtered_columns = ['venue.name', 'venue.categories', 'venue.location.lat', 'venue.location.lng']
nearby_venues =nearby_venues.loc[:, filtered_columns]

# filter the category for each row
nearby_venues['venue.categories'] = nearby_venues.apply(get_category_type, axis=1)

# clean columns
nearby_venues.columns = [col.split(".")[-1] for col in nearby_venues.columns]

nearby_venues.head()

,name,categories,lat,lng
0,Wendy’s,Fast Food Restaurant,43.807448,-79.199056


In [25]:
# Venues returned by Foursquare
print('{} venues were returned by Foursquare.'.format(nearby_venues.shape[0]))

1 venues were returned by Foursquare.


#### Repeat the same process to neighborhoods in Central Toronto.

In [26]:
CT_data = df5[df5['Borough'] == 'Central Toronto'].reset_index(drop=True)
CT_data.head(30)

,PostalCode,Borough,Neighbourhood,Latitude,Longitude
0,M4N,Central Toronto,Lawrence Park,43.728020,-79.388790
1,M4P,Central Toronto,Davisville North,43.712751,-79.390197
2,M4R,Central Toronto,North Toronto West,43.715383,-79.405678
3,M4S,Central Toronto,Davisville,43.704324,-79.388790
4,M4T,Central Toronto,"Moore Park, Summerhill East",43.689574,-79.383160
5,M4V,Central Toronto,"Summerhill West, Rathnelly, South Hill, Forest...",43.686412,-79.400049
6,M5N,Central Toronto,Roselawn,43.711695,-79.416936
7,M5P,Central Toronto,Forest Hill North & West,43.696948,-79.411307
8,M5R,Central Toronto,"The Annex, North Midtown, Yorkville",43.672710,-79.405678


In [27]:
def getNearbyVenues(names, latitudes, longitudes, radius=500):
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        print(name)
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
            
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Neighborhood', 
                  'Neighborhood Latitude', 
                  'Neighborhood Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)

In [28]:
Central_Toronto_venues = getNearbyVenues(names=CT_data['Neighbourhood'],
                                   latitudes=CT_data['Latitude'],
                                   longitudes=CT_data['Longitude']
                                  )

Lawrence Park
Davisville North
North Toronto West
Davisville
Moore Park, Summerhill East
Summerhill West, Rathnelly, South Hill, Forest Hill SE, Deer Park
Roselawn
Forest Hill North & West
The Annex, North Midtown, Yorkville


In [29]:
print(Central_Toronto_venues.shape)
Central_Toronto_venues.head()

(111, 7)


,Neighborhood,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,Lawrence Park,43.728020,-79.388790,Lawrence Park Ravine,43.726963,-79.394382,Park
1,Lawrence Park,43.728020,-79.388790,Dim Sum Deluxe,43.726953,-79.394260,Dim Sum Restaurant
2,Lawrence Park,43.728020,-79.388790,Zodiac Swim School,43.728532,-79.382860,Swim School
3,Lawrence Park,43.728020,-79.388790,TTC Bus #162 - Lawrence-Donway,43.728026,-79.382805,Bus Line
4,Davisville North,43.712751,-79.390197,Homeway Restaurant & Brunch,43.712641,-79.391557,Breakfast Spot


#### Check how many venues were returned for each neighborhood in Central Toronto.

In [30]:
Central_Toronto_venues.groupby('Neighborhood').count()

,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
Neighborhood,,,,,,
Davisville,33,33,33,33,33,33
Davisville North,6,6,6,6,6,6
Forest Hill North & West,5,5,5,5,5,5
Lawrence Park,4,4,4,4,4,4
"Moore Park, Summerhill East",2,2,2,2,2,2
North Toronto West,21,21,21,21,21,21
Roselawn,1,1,1,1,1,1
"Summerhill West, Rathnelly, South Hill, Forest Hill SE, Deer Park",16,16,16,16,16,16
"The Annex, North Midtown, Yorkville",23,23,23,23,23,23


#### Find out how many unique categories can be curated from all the returned venues.

In [31]:
print('There are {} uniques venue categories.'.format(len(Central_Toronto_venues['Venue Category'].unique())))

There are 62 uniques venue categories.


#### Analyze each neighborhood.

In [32]:
# one hot encoding
Central_Toronto_onehot = pd.get_dummies(Central_Toronto_venues[['Venue Category']], prefix="", prefix_sep="")

# add neighborhood column back to dataframe
Central_Toronto_onehot['Neighborhood'] = Central_Toronto_venues['Neighborhood'] 

# move neighborhood column to the first column
fixed_columns = [Central_Toronto_onehot.columns[-1]] + list(Central_Toronto_onehot.columns[:-1])
manhattan_onehot = Central_Toronto_onehot[fixed_columns]

Central_Toronto_onehot.head()

,American Restaurant,BBQ Joint,Bagel Shop,Bank,Bar,Breakfast Spot,Brewery,Burger Joint,Bus Line,Café,Chinese Restaurant,Clothing Store,Coffee Shop,Cosmetics Shop,Costume Shop,Department Store,Dessert Shop,Dim Sum Restaurant,Diner,Donut Shop,Farmers Market,Fast Food Restaurant,Flower Shop,Food & Drink Shop,Fried Chicken Joint,Garden,Gas Station,Gourmet Shop,Greek Restaurant,Gym,Gym / Fitness Center,History Museum,Hotel,Indian Restaurant,Italian Restaurant,Jewelry Store,Light Rail Station,Liquor Store,Mexican Restaurant,Middle Eastern Restaurant,Park,Pharmacy,Pizza Place,Playground,Pub,Rental Car Location,Restaurant,Salon / Barbershop,Sandwich Place,Seafood Restaurant,Spa,Sporting Goods Shop,Sports Bar,Supermarket,Sushi Restaurant,Swim School,Thai Restaurant,Toy / Game Store,Trail,Vegetarian / Vegan Restaurant,Vietnamese Restaurant,Yoga Studio,Neighborhood
0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,Lawrence Park
1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,Lawrence Park
2,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,Lawrence Park
3,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,Lawrence Park
4,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,Davisville North


In [33]:
Central_Toronto_onehot.shape

(111, 63)

#### Group rows by neighborhood and by taking the mean of the frequency of occurrence of each category.

In [34]:
Central_Toronto_grouped = Central_Toronto_onehot.groupby('Neighborhood').mean().reset_index()
Central_Toronto_grouped

,Neighborhood,American Restaurant,BBQ Joint,Bagel Shop,Bank,Bar,Breakfast Spot,Brewery,Burger Joint,Bus Line,Café,Chinese Restaurant,Clothing Store,Coffee Shop,Cosmetics Shop,Costume Shop,Department Store,Dessert Shop,Dim Sum Restaurant,Diner,Donut Shop,Farmers Market,Fast Food Restaurant,Flower Shop,Food & Drink Shop,Fried Chicken Joint,Garden,Gas Station,Gourmet Shop,Greek Restaurant,Gym,Gym / Fitness Center,History Museum,Hotel,Indian Restaurant,Italian Restaurant,Jewelry Store,Light Rail Station,Liquor Store,Mexican Restaurant,Middle Eastern Restaurant,Park,Pharmacy,Pizza Place,Playground,Pub,Rental Car Location,Restaurant,Salon / Barbershop,Sandwich Place,Seafood Restaurant,Spa,Sporting Goods Shop,Sports Bar,Supermarket,Sushi Restaurant,Swim School,Thai Restaurant,Toy / Game Store,Trail,Vegetarian / Vegan Restaurant,Vietnamese Restaurant,Yoga Studio
0,Davisville,0.000000,0.000000,0.000000,0.0000,0.030303,0.000000,0.030303,0.000000,0.00,0.060606,0.000000,0.000000,0.060606,0.000000,0.030303,0.000000,0.090909,0.00,0.030303,0.000000,0.030303,0.000000,0.000000,0.000000,0.0000,0.0,0.030303,0.030303,0.030303,0.060606,0.000000,0.000000,0.000000,0.030303,0.060606,0.0,0.0000,0.000000,0.000000,0.000000,0.030303,0.030303,0.060606,0.0,0.000000,0.000000,0.030303,0.000000,0.090909,0.030303,0.000000,0.000000,0.0000,0.0000,0.060606,0.00,0.030303,0.030303,0.0,0.000000,0.0000,0.000000
1,Davisville North,0.000000,0.000000,0.000000,0.0000,0.000000,0.166667,0.000000,0.000000,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.166667,0.000000,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.166667,0.0000,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.166667,0.000000,0.000000,0.0,0.0000,0.000000,0.000000,0.000000,0.166667,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.166667,0.000000,0.000000,0.000000,0.0000,0.0000,0.000000,0.00,0.000000,0.000000,0.0,0.000000,0.0000,0.000000
2,Forest Hill North & West,0.000000,0.000000,0.000000,0.0000,0.000000,0.000000,0.000000,0.000000,0.20,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0000,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.2,0.0000,0.000000,0.000000,0.000000,0.200000,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0000,0.0000,0.200000,0.00,0.000000,0.000000,0.2,0.000000,0.0000,0.000000
3,Lawrence Park,0.000000,0.000000,0.000000,0.0000,0.000000,0.000000,0.000000,0.000000,0.25,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.25,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0000,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.0000,0.000000,0.000000,0.000000,0.250000,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0000,0.0000,0.000000,0.25,0.000000,0.000000,0.0,0.000000,0.0000,0.000000
4,"Moore Park, Summerhill East",0.000000,0.000000,0.000000,0.0000,0.000000,0.000000,0.000000,0.000000,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0000,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.0000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.5,0.000000,0.000000,0.500000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0000,0.0000,0.000000,0.00,0.000000,0.000000,0.0,0.000000,0.0000,0.000000
5,North Toronto West,0.000000,0.000000,0.047619,0.0000,0.000000,0.000000,0.000000,0.000000,0.00,0.047619,0.047619,0.190476,0.095238,0.000000,0.000000,0.000000,0.047619,0.00,0.047619,0.000000,0.000000,0.047619,0.000000,0.000000,0.0000,0.0,0.000000,0.000000,0.000000,0.000000,0.047619,0.000000,0.000000,0.000000,0.000000,0.0,0.0000,0.000000,0.047619,0.000000,0.047619,0.000000,0.000000,0.0,0.000000,0.047619,0.047619,0.047619,0.000000,0.000000,0.047619,0.047

#### Confirm the new size.

In [35]:
Central_Toronto_grouped.shape

(9, 63)

Print each neighborhood along with the top 5 most common venues

In [36]:
num_top_venues = 5

for hood in Central_Toronto_grouped['Neighborhood']:
    print("----"+hood+"----")
    temp = Central_Toronto_grouped[Central_Toronto_grouped['Neighborhood'] == hood].T.reset_index()
    temp.columns = ['venue','freq']
    temp = temp.iloc[1:]
    temp['freq'] = temp['freq'].astype(float)
    temp = temp.round({'freq': 2})
    print(temp.sort_values('freq', ascending=False).reset_index(drop=True).head(num_top_venues))
    print('\n')

----Davisville----
                venue  freq
0      Sandwich Place  0.09
1        Dessert Shop  0.09
2         Pizza Place  0.06
3  Italian Restaurant  0.06
4                 Gym  0.06


----Davisville North----
               venue  freq
0     Sandwich Place  0.17
1     Breakfast Spot  0.17
2               Park  0.17
3  Food & Drink Shop  0.17
4   Department Store  0.17


----Forest Hill North & West----
              venue  freq
0          Bus Line   0.2
1     Jewelry Store   0.2
2             Trail   0.2
3              Park   0.2
4  Sushi Restaurant   0.2


----Lawrence Park----
                 venue  freq
0             Bus Line  0.25
1                 Park  0.25
2          Swim School  0.25
3   Dim Sum Restaurant  0.25
4  Rental Car Location  0.00


----Moore Park, Summerhill East----
                 venue  freq
0           Restaurant   0.5
1           Playground   0.5
2  American Restaurant   0.0
3    Indian Restaurant   0.0
4   Italian Restaurant   0.0


----North Toronto Wes

#### Put that into a *pandas* dataframe.

#### First, write a function to sort the venues in descending order.

In [37]:
def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    
    return row_categories_sorted.index.values[0:num_top_venues]

#### Create the new dataframe and display the top 10 venues for each neighborhood.

In [38]:
num_top_venues = 10

indicators = ['st', 'nd', 'rd']

# create columns according to number of top venues
columns = ['Neighborhood']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))

# create a new dataframe
neighborhoods_venues_sorted = pd.DataFrame(columns=columns)
neighborhoods_venues_sorted['Neighborhood'] = Central_Toronto_grouped['Neighborhood']

for ind in np.arange(Central_Toronto_grouped.shape[0]):
    neighborhoods_venues_sorted.iloc[ind, 1:] = return_most_common_venues(Central_Toronto_grouped.iloc[ind, :], num_top_venues)

neighborhoods_venues_sorted.head()

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Davisville,Sandwich Place,Dessert Shop,Pizza Place,Italian Restaurant,Coffee Shop,Sushi Restaurant,Café,Gym,Park,Pharmacy
1,Davisville North,Hotel,Sandwich Place,Breakfast Spot,Food & Drink Shop,Department Store,Park,Yoga Studio,Donut Shop,Dim Sum Restaurant,Diner
2,Forest Hill North & West,Jewelry Store,Trail,Park,Sushi Restaurant,Bus Line,Fast Food Restaurant,Dim Sum Restaurant,Diner,Donut Shop,Farmers Market
3,Lawrence Park,Dim Sum Restaurant,Park,Swim School,Bus Line,Yoga Studio,Fast Food Restaurant,Diner,Donut Shop,Farmers Market,Flower Shop
4,"Moore Park, Summerhill East",Playground,Restaurant,Yoga Studio,Department Store,Gourmet Shop,Gas Station,Garden,Fried Chicken Joint,Food & Drink Shop,Flower Shop


### Cluster Neighborhoods.

#### Run *k*-means to cluster the neighborhood into 5 clusters.

In [39]:
# set number of clusters
kclusters = 5

Central_Toronto_grouped_clustering = Central_Toronto_grouped.drop('Neighborhood', 1)

# run k-means clustering
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(Central_Toronto_grouped_clustering)

# check cluster labels generated for each row in the dataframe
kmeans.labels_[0:10]

array([0, 4, 2, 2, 3, 0, 1, 0, 0])

#### Create a new dataframe that includes the cluster as well as the top 10 venues for each neighborhood.

In [40]:
# add clustering labels
neighborhoods_venues_sorted.insert(0, 'Cluster Labels', kmeans.labels_)

In [41]:
neighborhoods_venues_sorted.head()

,Cluster Labels,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,0,Davisville,Sandwich Place,Dessert Shop,Pizza Place,Italian Restaurant,Coffee Shop,Sushi Restaurant,Café,Gym,Park,Pharmacy
1,4,Davisville North,Hotel,Sandwich Place,Breakfast Spot,Food & Drink Shop,Department Store,Park,Yoga Studio,Donut Shop,Dim Sum Restaurant,Diner
2,2,Forest Hill North & West,Jewelry Store,Trail,Park,Sushi Restaurant,Bus Line,Fast Food Restaurant,Dim Sum Restaurant,Diner,Donut Shop,Farmers Market
3,2,Lawrence Park,Dim Sum Restaurant,Park,Swim School,Bus Line,Yoga Studio,Fast Food Restaurant,Diner,Donut Shop,Farmers Market,Flower Shop
4,3,"Moore Park, Summerhill East",Playground,Restaurant,Yoga Studio,Department Store,Gourmet Shop,Gas Station,Garden,Fried Chicken Joint,Food & Drink Shop,Flower Shop


In [42]:
Central_Toronto_merged = CT_data

# merge Central_Toronto_grouped with Central_Toronto_data to add latitude/longitude for each neighborhood
Central_Toronto_merged = Central_Toronto_merged.join(neighborhoods_venues_sorted.set_index('Neighborhood'), on='Neighbourhood')

Central_Toronto_merged.head()

,PostalCode,Borough,Neighbourhood,Latitude,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,M4N,Central Toronto,Lawrence Park,43.728020,-79.388790,2,Dim Sum Restaurant,Park,Swim School,Bus Line,Yoga Studio,Fast Food Restaurant,Diner,Donut Shop,Farmers Market,Flower Shop
1,M4P,Central Toronto,Davisville North,43.712751,-79.390197,4,Hotel,Sandwich Place,Breakfast Spot,Food & Drink Shop,Department Store,Park,Yoga Studio,Donut Shop,Dim Sum Restaurant,Diner
2,M4R,Central Toronto,North Toronto West,43.715383,-79.405678,0,Clothing Store,Coffee Shop,Yoga Studio,Salon / Barbershop,Bagel Shop,Café,Chinese Restaurant,Dessert Shop,Diner,Fast Food Restaurant
3,M4S,Central Toronto,Davisville,43.704324,-79.388790,0,Sandwich Place,Dessert Shop,Pizza Place,Italian Restaurant,Coffee Shop,Sushi Restaurant,Café,Gym,Park,Pharmacy
4,M4T,Central Toronto,"Moore Park, Summerhill East",43.689574,-79.383160,3,Playground,Restaurant,Yoga Studio,Department Store,Gourmet Shop,Gas Station,Garden,Fried Chicken Joint,Food & Drink Shop,Flower Shop


#### Visualize the resulting clusters.

In [43]:
# create map
map_clusters = folium.Map(location=[latitude, longitude], zoom_start=11)

# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i + x + (i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(Central_Toronto_merged['Latitude'], Central_Toronto_merged['Longitude'], Central_Toronto_merged['Neighbourhood'], Central_Toronto_merged['Cluster Labels']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters